In [1]:
%matplotlib inline
import numpy as np
import tensorflow as tf
import matplotlib.pyplot as plt
from sklearn.model_selection import train_test_split

In [ ]:
np.random.seed(12)
num_observations = 5000

x1 = np.random.multivariate_normal([0, 0], [[2, .75],[.75, 2]], num_observations)
x2 = np.random.multivariate_normal([1, 4], [[1, .75],[.75, 1]], num_observations)
x3 = np.random.multivariate_normal([2, 8], [[0, .75],[.75, 0]], num_observations)

simulated_separableish_features = np.vstack((x1, x2, x3)).astype(np.float32)
print (simulated_separableish_features.shape)
print (simulated_separableish_features)
simulated_labels = np.hstack((np.zeros(num_observations),
                                np.ones(num_observations), np.ones(num_observations) + 1))
print (simulated_labels.shape)
print (simulated_labels)

plt.figure(figsize=(12,8))
plt.scatter(simulated_separableish_features[:, 0], simulated_separableish_features[:, 1],
            c = simulated_labels, alpha = .4)

In [ ]:
# Reformat the input labels to one-hot encoded vector
labels_onehot = np.zeros((simulated_labels.shape[0], 3)).astype(int)
labels_onehot[np.arange(len(simulated_labels)), simulated_labels.astype(int)] = 1

train_dataset, test_dataset, \
train_labels, test_labels = train_test_split(
    simulated_separableish_features, labels_onehot, test_size = .1, random_state = 12)
print (train_dataset.shape)
print (train_dataset)
print (test_dataset.shape)
print (test_dataset)
print (train_labels.shape)
print (train_labels)
print (test_labels.shape)
print (test_labels)


In [2]:
# in this case; num_features = num_statements
# num_input = num_test_cases
dataset = np.matrix([
    [1,1,1,1,0,1,0,0,1],
    [1,0,0,0,1,1,1,1,0],
    [0,0,0,0,0,1,1,0,0],
    [1,1,0,0,1,0,1,1,1],
    [1,1,1,0,1,1,1,1,1],
    [0,0,1,0,0,1,1,1,0],
    [1,1,1,1,0,1,0,1,1]
]).astype(int)
# in labels, 0 means success, 1 means failure
#labels = np.array([[0],[0],[0],[0],[0],[1],[1]])
labels = np.array([0,0,0,0,0,1,1])
# transform the labels to one-hot format
labels_onehot = np.zeros((labels.shape[0], 2)).astype(int)
labels_onehot[np.arange(len(labels)), labels.astype(int)] = 1
# divide the dataset into train and test datasets
train_dataset, test_dataset, \
train_labels, test_labels = train_test_split(
    dataset, labels, test_size = .1, random_state = 12)
print (train_labels)
print (train_dataset)

[1 0 0 0 1 0]
[[0 0 1 0 0 1 1 1 0]
 [1 1 1 1 0 1 0 0 1]
 [0 0 0 0 0 1 1 0 0]
 [1 0 0 0 1 1 1 1 0]
 [1 1 1 1 0 1 0 1 1]
 [1 1 0 0 1 0 1 1 1]]


In [32]:
# Build_model
hidden_nodes = 5
# num_labels is 1, only true/false classification
num_labels = 1
batch_size = 100
# num_features is num_statements in this case
num_features = train_dataset.shape[1]
learning_rate = .01

graph = tf.Graph()
with graph.as_default():
    
    # Data
    tf_train_dataset = tf.placeholder(tf.float32, shape = [None, num_features])
    tf_train_labels = tf.placeholder(tf.float32, None)
    tf_test_dataset = tf.constant(test_dataset, dtype=tf.float32)
  
    # Weights and Biases
    layer1_weights = tf.Variable(tf.truncated_normal([num_features, hidden_nodes]))
    layer1_biases = tf.Variable(tf.zeros([hidden_nodes]))
    
    layer2_weights = tf.Variable(tf.truncated_normal([hidden_nodes, num_labels]))
    layer2_biases = tf.Variable(tf.zeros([num_labels]))
    
    # Three-Layer Network
    def three_layer_network(data):
        input_layer = tf.matmul(data, layer1_weights)
        hidden = input_layer + layer1_biases
        output_layer = tf.nn.sigmoid(tf.matmul(hidden, layer2_weights) + layer2_biases)
        return output_layer
    
    # Model Scores
    model_scores = three_layer_network(tf_train_dataset)
    model_scores_t = tf.reshape(model_scores,[-1])
    
    # Loss
    # cross_entropy = tf.reduce_sum(- tf_train_labels * tf.log(model_scores_t))
    cross_entropy = tf.reduce_sum(tf.square(tf_train_labels - model_scores_t))
    loss = tf.reduce_mean(cross_entropy)
    #loss = tf.nn.softmax_cross_entropy_with_logits(logits=model_scores_t, labels=tf_train_labels)
  
    # Optimizer
    optimizer = tf.train.GradientDescentOptimizer(learning_rate).minimize(loss)
    
    # Predictions
    #train_prediction = tf.nn.sigmoid(model_scores)
    train_prediction = model_scores
    #test_prediction = tf.nn.sigmoid(three_layer_network(tf_test_dataset))
    test_prediction = tf.reshape(three_layer_network(tf_test_dataset),[-1])

In [4]:
# PART B
# estimate the suspiciousness of each statement
test_susp_dataset = np.identity(dataset.shape[1])
print (test_susp_dataset)
print (test_susp_dataset.shape)

[[ 1.  0.  0.  0.  0.  0.  0.  0.  0.]
 [ 0.  1.  0.  0.  0.  0.  0.  0.  0.]
 [ 0.  0.  1.  0.  0.  0.  0.  0.  0.]
 [ 0.  0.  0.  1.  0.  0.  0.  0.  0.]
 [ 0.  0.  0.  0.  1.  0.  0.  0.  0.]
 [ 0.  0.  0.  0.  0.  1.  0.  0.  0.]
 [ 0.  0.  0.  0.  0.  0.  1.  0.  0.]
 [ 0.  0.  0.  0.  0.  0.  0.  1.  0.]
 [ 0.  0.  0.  0.  0.  0.  0.  0.  1.]]
(9, 9)


In [5]:
def accuracy(predictions, labels):
    preds_correct_boolean =  np.argmax(predictions, 1) == np.argmax(labels, 1)
    correct_predictions = np.sum(preds_correct_boolean)
    accuracy = 100.0 * correct_predictions / predictions.shape[0]
    return accuracy

In [35]:
num_steps = 10001

with tf.Session(graph=graph) as session:
    tf.initialize_all_variables().run()
    for step in range(num_steps):
        offset = (step * batch_size) % (train_labels.shape[0] - batch_size)
        minibatch_data = train_dataset[offset:(offset + batch_size), :]
        minibatch_labels = train_labels[offset:(offset + batch_size)]
        feed_dict = {tf_train_dataset : minibatch_data, tf_train_labels : minibatch_labels}
        #session.run(tf.global_variables_initializer())
        
        _, l, predictions = session.run(
            [optimizer, loss, model_scores_t], feed_dict = feed_dict
            )
        
        if step % 1000 == 0:
            # print (session.run(train_prediction))
            print 'Minibatch loss at step {0}: {1} - {2}'.format(step, l, predictions)

    print test_prediction.eval()
    print test_labels
    # print 'Test accuracy: {0}%'.format(accuracy(test_prediction.eval(), test_labels))
    for count in range(dataset.shape[1]):
        print (count)
        print (test_susp_dataset[count,:].reshape(1,dataset.shape[1]))
        feed_dict = {tf_test_dataset: test_susp_dataset[count,:].reshape(1,dataset.shape[1])}
        predictions = session.run([test_prediction], feed_dict)
        print (predictions)

Minibatch loss at step 0: 1.45251369476 - [ 0.4408195   0.04764955  0.32713005  0.36614913  0.07790515  0.21499357]
Minibatch loss at step 1000: 0.0184685122222 - [ 0.95740771  0.08134051  0.02175828  0.04942956  0.91748554  0.01768586]
Minibatch loss at step 2000: 0.0071024550125 - [ 0.97555315  0.05149345  0.01090244  0.03074255  0.94817978  0.01019452]
Minibatch loss at step 3000: 0.00416782777756 - [ 0.98194075  0.03976906  0.00742066  0.02357069  0.96010703  0.00761707]
Minibatch loss at step 4000: 0.00288299331442 - [ 0.98530072  0.03316745  0.00568622  0.0195851   0.96665514  0.00624943]
Minibatch loss at step 5000: 0.00217740796506 - [ 0.98742491  0.02891739  0.00464405  0.01701338  0.9709664   0.00539327]
Minibatch loss at step 6000: 0.000698460731655 - [ 0.97400969  0.00479215]
Minibatch loss at step 7000: 0.00143625540659 - [ 0.99000311  0.02356232  0.00344051  0.01380553  0.97634     0.00434843]
Minibatch loss at step 8000: 0.00122020719573 - [ 0.99085581  0.02173358  0.003

In [22]:
a = tf.constant([[0.9],[0.4],[0.4]])
anew = tf.constant([0.9,0.4,0.4])
b = tf.constant([[1],[0],[0]])
bnew = tf.constant([1,0,0],dtype=tf.float32)
result =  tf.nn.softmax_cross_entropy_with_logits(logits=a,labels=b,dim=0)[0]
result2 = tf.nn.sigmoid([[1.2],[0.9],[0.4]])
result3 = tf.reduce_sum(-bnew*tf.log(anew))
sess = tf.InteractiveSession()
sess.run(tf.initialize_all_variables())
print result.eval()
print result2.eval()
print result3.eval()

anew = tf.reshape(a,[-1])
print anew.eval()

0.794377
[[ 0.76852483]
 [ 0.71094948]
 [ 0.59868765]]
0.105361
[ 0.89999998  0.40000001  0.40000001]
